Face Detection using OpenCV

In [20]:
from tkinter import *
from PIL import Image, ImageTk
import cv2
import numpy as np

root = Tk()
root.title("Face Detection")
root.geometry("626x417")
root.resizable(0, 0)

# Load pre-trained face detection model
net = cv2.dnn.readNetFromCaffe('deploy.prototxt', 'res10_300x300_ssd_iter_140000.caffemodel')


# Global variables for video capture and processing
vid = None
is_canny_active = False
is_sobel_active = False
is_kalman_active = False
is_hough_active = False
is_cnn_active = False
kalman = cv2.KalmanFilter(4, 2)  # Initialize Kalman filter

# Function to start the camera
def start_camera():
    global vid
    vid = cv2.VideoCapture(0)
    update_camera()

# Function to update the camera feed
def update_camera():
    global vid
    ret, frame = vid.read()
    if ret:
        frame = cv2.resize(frame, (288, 216))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
        
        if is_canny_active:
            frame = detect_faces_canny(frame)
        elif is_sobel_active:
            frame = detect_faces_sobel(frame)
        elif is_kalman_active:
            frame = detect_and_track_faces(frame)
        elif is_hough_active:
            frame = detect_faces_hough(frame)
        elif is_cnn_active:
            frame = detect_faces_cnn(frame)
        elif is_all_in_one_active:
            frame = detect_faces_all_in_one(frame)
        
        frame_pil = Image.fromarray(frame)
        frame_tk = ImageTk.PhotoImage(frame_pil)
        camera_module_label.configure(image=frame_tk)
        camera_module_label.image = frame_tk
        camera_module_label.after(10, update_camera)

# Function to detect faces using multiple techniques combined (Haar cascade, CNN, Kalman filter)
def detect_faces_all_in_one(frame):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    
    # Apply Haar cascade classifier
    faces_haar = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Apply CNN face detection
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    
    # Apply Kalman filter
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            center = np.array([startX + (endX - startX)/2, startY + (endY - startY)/2], dtype=np.float32).reshape(2, 1)
            prediction = kalman.predict()
            measurement = np.array([[np.float32(center[0])], [np.float32(center[1])]])
            kalman.correct(measurement)
    
    # Draw rectangles and write "face" on the detected areas
    for (x, y, w, h) in faces_haar:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, 'Face', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.putText(frame, 'Face', (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    return frame

# Function to toggle Canny edge detection
def canny_edge():
    global is_canny_active, is_sobel_active, is_kalman_active, is_hough_active, is_cnn_active
    is_canny_active = not is_canny_active
    if is_canny_active:
        is_sobel_active = False
        is_kalman_active = False
        is_hough_active = False
        is_cnn_active = False

# Function to toggle Sobel edge detection
def sobel_edge():
    global is_sobel_active, is_canny_active, is_kalman_active, is_hough_active, is_cnn_active
    is_sobel_active = not is_sobel_active
    if is_sobel_active:
        is_canny_active = False
        is_kalman_active = False
        is_hough_active = False
        is_cnn_active = False

# Function to toggle Kalman Filter
def kalman_filter():
    global is_kalman_active, is_canny_active, is_sobel_active, is_hough_active, is_cnn_active
    is_kalman_active = not is_kalman_active
    if is_kalman_active:
        is_canny_active = False
        is_sobel_active = False
        is_hough_active = False
        is_cnn_active = False

# Function to toggle Hough Transform
def hough_transform():
    global is_hough_active, is_canny_active, is_sobel_active, is_kalman_active, is_cnn_active
    is_hough_active = not is_hough_active
    if is_hough_active:
        is_canny_active = False
        is_sobel_active = False
        is_kalman_active = False
        is_cnn_active = False

# Function to toggle CNN
def cnn():
    global is_cnn_active, is_canny_active, is_sobel_active, is_kalman_active, is_hough_active
    is_cnn_active = not is_cnn_active
    if is_cnn_active:
        is_canny_active = False
        is_sobel_active = False
        is_kalman_active = False
        is_hough_active = False

# Function to detect faces using Haar cascade classifier and apply Canny edge detection
def detect_faces_canny(frame):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    
    # Detect faces using Haar cascade classifier
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Apply Canny edge detection to the entire frame
    edges = cv2.Canny(gray, 100, 200)
    
    # Create a mask where edges are detected
    mask = edges == 255
    
    # Assign green color to the pixels where edges are detected
    frame[mask] = [0, 255, 0]
    
    # Draw rectangles around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    return frame

# Function to detect faces using Haar cascade classifier and apply Sobel edge detection
def detect_faces_sobel(frame):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    
    # Detect faces using Haar cascade classifier
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Apply Sobel edge detection to the entire frame
    edges_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    edges_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
    edges = cv2.convertScaleAbs(edges_x) + cv2.convertScaleAbs(edges_y)
    
    # Combine the edges detected in face regions and full frame
    frame[np.where((edges > 50))] = [0, 255, 0]
    
    # Draw rectangles around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    return frame

# Function to detect faces using Haar cascade classifier and apply Kalman filter
def detect_and_track_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Get the center of the face bounding box
        center = np.array([x + w/2, y + h/2], dtype=np.float32).reshape(2, 1)
        
        # Predict the next position of the face using Kalman filter
        prediction = kalman.predict()
        
        # Update the Kalman filter with the measured face position
        measurement = np.array([[np.float32(center[0])], [np.float32(center[1])]])
        kalman.correct(measurement)
        
        # Draw a dot at the predicted position of the face
        cv2.circle(frame, (int(prediction[0]), int(prediction[1])), 4, (0, 255, 0), -1)
    
    return frame

# Function to detect faces using Haar cascade classifier and apply Hough circle transform
def detect_faces_hough(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Get the face region
        face_region = frame[y:y+h, x:x+w]
        gray_face = cv2.cvtColor(face_region, cv2.COLOR_BGR2GRAY)
        
        # Apply Hough circle transform
        circles = cv2.HoughCircles(gray_face, cv2.HOUGH_GRADIENT, dp=1, minDist=20,
                                   param1=50, param2=30, minRadius=0, maxRadius=0)
        
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for circle in circles[0, :]:
                center = (circle[0], circle[1])
                radius = circle[2]
                # Draw the circle on the frame
                cv2.circle(frame, (x + center[0], y + center[1]), radius, (0, 255, 0), 2)

    return frame

# Function to detect faces using pre-trained CNN model
def detect_faces_cnn(frame):
    h, w = frame.shape[:2]

    # Create blob from input frame and perform forward pass
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # Loop over the detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        
        # Filter out weak detections
        if confidence > 0.5:
            # Extract bounding box coordinates
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            # Draw rectangle around the face
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 0, 0), 2)

    return frame

# Load pre-trained face detection classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Background and labels setup
background_image = PhotoImage(file="background_face.png")
background_label = Label(root, image=background_image)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

Title_label = Label(root, text="Face Detection using Open CV", font="Elephant 15 bold", bg='#e0ae8d', fg='white')
Title_label.place(x=140, y=0)

app_icon = PhotoImage(file="face.png")
root.iconphoto(False, app_icon)
app_logo = PhotoImage(file="Open_CV.png")
app_logo_resize = app_logo.subsample(5, 5)
app_logo_label = Label(root, image=app_logo_resize, bg='#e0ae8d')
app_logo_label.place(x=78, y=0)

camera_module = PhotoImage(file="camera_module.png")
camera_module_resize = camera_module.subsample(2, 2)
camera_module_label = Label(root, image=camera_module_resize, bg='#40857f', border=3)
camera_module_label.place(x=10, y=96)

start_button = Button(root, text="Start Camera", font="Elephant 10", bg='#f5922d', activebackground='#f57e2d',
                      fg='#202930', border=2, command=start_camera)
start_button.place(x=108, y=350)

ce_button = Button(root, text="CED", font="Elephant 10", bg='#f5922d', activebackground='#f57e2d', fg='#202930',
                   border=2, command=canny_edge)
ce_button.place(x=321, y=100)

se_button = Button(root, text="SED", font="Elephant 10", bg='#f5922d', activebackground='#f57e2d', fg='#202930',
                   border=2, command=sobel_edge)
se_button.place(x=320, y=138)

kf_button = Button(root, text="KFR", font="Elephant 10", bg='#f5922d', activebackground='#f57e2d', fg='#202930',
                   border=2, command=kalman_filter)
kf_button.place(x=319, y=178)

ht_button = Button(root, text="HTR", font="Elephant 10", bg='#f5922d', activebackground='#f57e2d', fg='#202930',
                   border=2, command=hough_transform)
ht_button.place(x=319, y=215)

cnn_button = Button(root, text="CNN", font="Elephant 10", bg='#f5922d', activebackground='#f57e2d', fg='#202930',
                    border=2, command=cnn)
cnn_button.place(x=320, y=253)

# Define a global variable to track the state of all_in_one function
is_all_in_one_active = False

def all_in_one_toggle():
    global is_all_in_one_active
    if is_all_in_one_active:
        is_all_in_one_active = False
    else:
        is_all_in_one_active = True
        all_in_one()

# Function to toggle All-in-One face detection mode
def all_in_one():
    global is_all_in_one_active
    is_all_in_one_active = not is_all_in_one_active
    if is_all_in_one_active:
        all_button.config(text="STOP", bg='#ff6347')  # Change button text and color
    else:
        all_button.config(text="AAM", bg='#f5922d')  # Change button text and color

# AAM button
all_button = Button(root, text="AAM", font="Elephant 10", bg='#f5922d', activebackground='#f57e2d', fg='#202930',
                    border=2, command=all_in_one)
all_button.place(x=320, y=292)

root.mainloop()


C:\Users\KIIT\AppData\Local\Temp\ipykernel_26008\3969351577.py:212: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.circle(frame, (int(prediction[0]), int(prediction[1])), 4, (0, 255, 0), -1)
